----

# **Experimentation for End to end Generative AI Project: MCQ Generator using OpenAI, Langchain Streamlit**

---------

#### **Importing Required Libraries**

In [14]:
import pandas as pd
import PyPDF2
import json
import traceback
import os
from dotenv import load_dotenv

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback 


#### **Setting up OPENAI API KEY**

In [15]:
load_dotenv() # load environment variables from .env file 

openai_api_key = os.getenv('OPENAI_API_KEY') # get api key from environment variable

#### **Create an instance of LLM with api key and model name**

In [16]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo",temperature=0.5) 

- The provided code snippet initializes an instance of the ChatOpenAI class from the openai library, configured with an OpenAI API key, the "gpt-3.5-turbo" model, and a temperature parameter of 0.5.

#### **Define Prompt Template**

In [21]:
# sample response json for prompt template to generate quiz

RESPONSE_JSON = { 
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

#### **Prompt template to generate quiz from text**

In [22]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [23]:

quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

#### **Define llm chain to generate quiz**

In [24]:
quiz_chain = LLMChain(
    llm = llm,
    prompt = quiz_generation_prompt,
    output_key = "quiz",
    verbose = True
)

#### **Prompt template2 to generate quiz from text**

In [25]:
# prompt template2 to generate quiz from text

TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

#### **This prompt template is used to evaluate the complexity of the quiz**

In [26]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE) 

#### **Review chain to evaluate the quiz and give feedback**

In [27]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True) 

#### **Sequential chain to generate quiz and evaluate it**

In [29]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)  